# SIP Trace parser

Just Basic Call Flow for now

In [2]:
import re

In [3]:
#log_file = '/home/bibo/.blink/logs/sip_trace.txt'
log_file = 'sip_trace.txt'

In [4]:
f = open(log_file)
log = f.read()
log_chunks = log.split('--')

In [5]:
pat_ip = re.compile('^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}')
pat_date = re.compile('^\d+-\d+-\d+')
ip_flag = False
chunk_dict = {}
log_list = []

In [6]:
for chunk in log_chunks:
    for line in chunk.splitlines():
        if pat_date.search(line):
            continue
        if line.startswith('<'):
            break
        elif pat_ip.search(line):
            chunk_dict['flow_header'] = line
        if ':' in line:
            d = line.split(':', 1)
            chunk_dict[d[0]] = d[1]
        elif ' ' in line:
            d = line.split(' ', 1)
            chunk_dict[d[0]] = d[1]
        else:
            continue
    log_list.append(chunk_dict)
    chunk_dict = {}

In [7]:
flow_id_list = []
for flow_dict in log_list:
    if 'Call-ID' in flow_dict.keys():
        flow_id_list.append(flow_dict)

In [8]:
def append_flow_to_dict(flows_dict, call_id, flow):
    if not call_id in flows_dict:
            flows_dict[call_id] = [flow]
    else:
        flows_dict[call_id] += [flow]

In [9]:
flows_by_call = {}

for flow in log_list:
    call_id = flow.get("Call-ID")
    if call_id:
        append_flow_to_dict(flows_by_call, call_id, flow)

In [10]:
reg_flows_by_call = {}
invite_flows_by_call = {}
for call_id in flows_by_call.keys():
    for flow in flows_by_call[call_id]:
        if 'REGISTER' in flow['CSeq']:
            append_flow_to_dict(reg_flows_by_call, call_id, flow)
        if 'INVITE' in flow['CSeq']:
            append_flow_to_dict(invite_flows_by_call, call_id, flow)

In [11]:
for call_id in reg_flows_by_call.keys():
    print('New Registration')
    for flow in reg_flows_by_call[call_id]:
        print(flow['flow_header'])
        print('    |                                               |')
        if 'SIP/2.0' in flow.keys():
            print('    |                  ' + flow['SIP/2.0'] + '             |')
            print('    |---------------------------------------------->|')
        if 'REGISTER sip' in flow.keys():
            print('    |          ' + flow['CSeq'] + '                          |')
            print('    |---------------------------------------------->|')
            print()
    print()

New Registration
192.168.8.104:36283 -(SIP over TLS)-> 81.23.228.129:443
    |                                               |
    |           1 REGISTER                          |
    |---------------------------------------------->|

81.23.228.129:443 -(SIP over TLS)-> 192.168.8.104:36283
    |                                               |
    |                  401 Unauthorized             |
    |---------------------------------------------->|
192.168.8.104:36283 -(SIP over TLS)-> 81.23.228.129:443
    |                                               |
    |           2 REGISTER                          |
    |---------------------------------------------->|

81.23.228.129:443 -(SIP over TLS)-> 192.168.8.104:36283
    |                                               |
    |                  200 OK             |
    |---------------------------------------------->|

New Registration
192.168.8.104:36283 -(SIP over TLS)-> 81.23.228.129:443
    |                                        

In [12]:
for call_id in invite_flows_by_call.keys():
    print('New Session')
    for flow in invite_flows_by_call[call_id]:
        print(flow['flow_header'])
        print('    |                                               |')
        if 'SIP/2.0' in flow.keys():
            print('    |        ' + flow['SIP/2.0'] + '          |')
            print('    |---------------------------------------------->|')
        else:
            print('    |          ' + flow['CSeq'] + '                          |')
            print('    |---------------------------------------------->|')
            print()
    print()

New Session
192.168.8.104:45197 -(SIP over TLS)-> 81.23.228.129:443
    |                                               |
    |           8862 INVITE                          |
    |---------------------------------------------->|

81.23.228.129:443 -(SIP over TLS)-> 192.168.8.104:45197
    |                                               |
    |        407 Proxy Authentication Required          |
    |---------------------------------------------->|
192.168.8.104:45197 -(SIP over TLS)-> 81.23.228.129:443
    |                                               |
    |           8863 INVITE                          |
    |---------------------------------------------->|

81.23.228.129:443 -(SIP over TLS)-> 192.168.8.104:45197
    |                                               |
    |        100 Trying          |
    |---------------------------------------------->|
81.23.228.129:443 -(SIP over TLS)-> 192.168.8.104:45197
    |                                               |
    |        100 